transforms.Compose()はpytorchで画像を指定したデータ形式に変換するためのメソッドです。ToTensor()を使用することで、取り込んだMNIST画像（最初はnumpy配列）をテンソルへと変換します。

transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))で各チャンネルにおける、テンソル（画像）の輝度値を平均0.5、標準偏差0.5になるように標準化します。今だったら3チャンネル分をまとめて指定します。

最初にdiscriminator.zero_grad()で勾配の初期値を0に設定します。その後は。トレイン画像とgeneratorが生成したフェイク画像それぞれについて損失を定義し、スコアを出力します。

それぞれ別々にスコアを計算しなければいけない理由は、前回のテキストのLossをみてもらえればわかると思います。Lossの式が第一項と第二項に別れているのはそのためです。

backward()メソッドはpytorchのtorch.Tensorの標準的なクラスであり、backward()を呼び出すだけで全ての勾配を自動的に計算させることが可能です。そしてstepメソッドで更新させ

In [17]:
# 保存用のdirectory作成
#!mkdir results

In [1]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torch.nn as nn
from torch.autograd import Variable
import itertools
import math
import time
import matplotlib.pyplot as plt
from IPython import display
import torchvision.datasets as dsets
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
])

train_dataset = dsets.MNIST(root='~/tmp/mnis', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)



class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out = self.model(x.view(x.size(0), 784))
        out = out.view(out.size(0), -1)
        return out



class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
        nn.Linear(100,256),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Dropout(0.3),
        nn.Linear(256, 512),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Dropout(0.3),
        nn.Linear(512, 1024),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Dropout(0.3),
        nn.Linear(1024, 784),
        nn.Tanh()
        )
        
    def forward(self, x):
        x = x.view(x.size(0),100)
        out = self.model(x) # 100,784
        return out
        
discriminator = Discriminator()
generator = Generator()



criterion = nn.BCELoss()
lr = 0.0002
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)



def train_discriminator(discriminator, images, real_labels, fake_images, fake_labels):
    discriminator.zero_grad()
    outputs = discriminator(images)
    real_loss = criterion(outputs, real_labels)
    real_score = outputs
    
    outputs = discriminator(fake_images) 
    fake_loss = criterion(outputs, fake_labels)
    fake_score = outputs

    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss, real_score, fake_score

def train_generator(generator, real_labels, outputs):
    # 勾配を0に設定
    generator.zero_grad()
    # lossを設定
    g_loss = criterion(outputs, real_labels)
    # backwardを設定
    g_loss.backward()
    # optimizerを設定
    g_optimizer.step()
    # generatorのlossを返す
    return g_loss

# draw samples from the input distribution to inspect the generation on training 
num_test_samples = 16
test_noise = Variable(torch.randn(num_test_samples, 100))

# create figure for plotting
size_figure_grid = int(math.sqrt(num_test_samples))
fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(6, 6))
for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
    ax[i,j].get_xaxis().set_visible(False)
    ax[i,j].get_yaxis().set_visible(False)

# set number of epochs and initialize figure counter
num_epochs = 200
num_batches = len(train_loader)
num_fig = 0


for epoch in range(num_epochs):
    for n, (images, _) in enumerate(train_loader):
        images = Variable(images)
        real_labels = Variable(torch.ones(images.size(0)))
        
        # Sample from generator
        noise = Variable(torch.randn(images.size(0), 100))
        fake_images = generator(noise)
        fake_labels = Variable(torch.zeros(images.size(0)))
        
        # Train the discriminator
        d_loss, real_score, fake_score = train_discriminator(discriminator, images, real_labels, fake_images, fake_labels)
        
        # Sample again from the generator and get output from discriminator
        noise = Variable(torch.randn(images.size(0), 100))
        fake_images = generator(noise)
        outputs = discriminator(fake_images)

        # Train the generator
        g_loss = train_generator(generator,real_labels, outputs)
        
        if (n+1) % 100 == 0:
            test_images = generator(test_noise)
            
            for k in range(num_test_samples):
                i = k//4
                j = k%4
                ax[i,j].cla()
                ax[i,j].imshow(test_images[k,:].data.cpu().numpy().reshape(28, 28), cmap='Greys')
            display.clear_output(wait=True)
            display.display(plt.gcf())
            
            plt.savefig('results/mnist-gan-%03d.png'%num_fig)
            num_fig += 1
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, ' 
                  'D(x): %.2f, D(G(z)): %.2f' 
                  %(epoch + 1, num_epochs, n+1, num_batches, d_loss.data.item(), g_loss.data.item(),
                    real_score.data.mean(), fake_score.data.mean()))

fig.close()

<Figure size 600x600 with 16 Axes>

Epoch [200/200], Step[600/600], d_loss: 0.9687, g_loss: 1.5315, D(x): 0.68, D(G(z)): 0.35


AttributeError: 'Figure' object has no attribute 'close'

3.問題2 論文調査

GANは生成的モデルを訓練する新しい方法として導入された.GANはデータ分布を捉える生成モデルGと、標本データがGではなく学習データから得られる確率を推定する判別モデルDの2つの「敵対的」モデルで構成されている.
Gはデータxの分布を学習するためにノイズ分布からデータ空間の写像関数を学習する.
GとDの両方とも、多層パーセプトロンのような非線形写像関数である.
 GとDは両方とも同時に訓練されます.
G（1 - D（G（z）））を最小にするようにGのパラメータを調整し、L（D（X））を最小にするようにDのパラメータを調整する.それらが価値関数V（G、D）を持つ2人用のミニマックスゲームに従っているかのように見える.

GANのG,Dともに追加の情報yを入力することでconditionalモデルに拡張することができる.yは、クラスラベルや他のモダリティからのデータなど,あらゆる種類の補助情報とする.

One-Hot-Vectorとしてencodeされた,クラスラベルで条件付けをしたMNIST画像についてconditional敵対ネットを訓練した。ノイズは次元数100の一様分布.zとyは隠れ層にマッピングされ、ReLuの後,層数はそれぞれ200と1000である.次にマッピングする前にcombineして1200にする.最後にsigmoidを使用し784次元のMNISTをサンプル生成する.

Dはxを240単位と5個のmaxout [6]層に、yを50単位と5個のmaxout層にマッピングする.
両方の隠れ層は、シグモイド層に供給される前に,240単位と4個のジョイントマックスアウト層にマッピングされた.
ただし,Dは機能する場合,アーキテクチャはこの限りでなく,体感的にマックスアウトが良好であった.

サイズ100のミニバッチと初期学習率0.1の確率的勾配降下法を使用して訓練した.momentumは初期値0.5で0.7まで増加した.Dropoutは0.5でGとD両方に適用した.検証セットでの対数尤度の最善の推定値を停止点として使用した.

問題2で調査した、ConditionalGANを実装

In [6]:
'''
通常のGANのコード
'''

from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()



gan = GAN()
gan.train(epochs=30000, batch_size=32,sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

C:\Users\umini\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.592553, acc.: 84.38%] [G loss: 0.580163]
1 [D loss: 0.412094, acc.: 75.00%] [G loss: 0.731064]
2 [D loss: 0.322611, acc.: 82.81%] [G loss: 0.790731]
3 [D loss: 0.319819, acc.: 82.81%] [G loss: 1.063783]
4 [D loss: 0.209118, acc.: 95.31%] [G loss: 1.154427]
5 [D loss: 0.178178, acc.: 100.00%] [G loss: 1.330730]
6 [D loss: 0.170049, acc.: 98.44%] [G loss: 1.420514]
7 [D loss: 0.155218, acc.: 98.44%] [G loss: 1.636323]
8 [D loss: 0.143092, acc.: 100.00%] [G loss: 1.729054]
9 [D loss: 0.123009, acc.: 100.00%] [G loss: 1.831839]
10 [D loss: 0.122713, acc.: 100.00%] [G loss: 1.908354]
11 [D loss: 0.101911, acc.: 100.00%] [G loss: 1.960722]
12 [D loss: 0.081103, acc.: 100.00%] [G loss: 1.993467]
13 [D loss: 0.087393, acc.: 100.00%] [G loss: 2.269865]
14 [D loss: 0.079817, acc.: 100.00%] [G loss: 2.269819]
15 [D loss: 0.070076, acc.: 100.00%] [G loss: 2.343985]
16 [D loss: 0.075073, acc.: 100.00%] [G loss: 2.445027]
17 [D loss: 0.056224, acc.: 100.00%] [G loss: 2.511913]
18 [D los

152 [D loss: 0.207791, acc.: 95.31%] [G loss: 3.146133]
153 [D loss: 0.104597, acc.: 98.44%] [G loss: 3.959078]
154 [D loss: 0.237610, acc.: 87.50%] [G loss: 4.011675]
155 [D loss: 0.271163, acc.: 87.50%] [G loss: 4.093668]
156 [D loss: 0.497557, acc.: 75.00%] [G loss: 3.046835]
157 [D loss: 0.116762, acc.: 96.88%] [G loss: 4.109526]
158 [D loss: 0.781795, acc.: 70.31%] [G loss: 2.454415]
159 [D loss: 0.252430, acc.: 84.38%] [G loss: 3.454620]
160 [D loss: 0.100653, acc.: 96.88%] [G loss: 3.824025]
161 [D loss: 0.207041, acc.: 95.31%] [G loss: 3.611552]
162 [D loss: 0.269944, acc.: 89.06%] [G loss: 4.090637]
163 [D loss: 0.484561, acc.: 76.56%] [G loss: 3.098317]
164 [D loss: 0.142430, acc.: 93.75%] [G loss: 3.516989]
165 [D loss: 0.361651, acc.: 87.50%] [G loss: 3.152167]
166 [D loss: 0.311344, acc.: 82.81%] [G loss: 3.895805]
167 [D loss: 0.570953, acc.: 76.56%] [G loss: 2.815182]
168 [D loss: 0.191892, acc.: 93.75%] [G loss: 3.340853]
169 [D loss: 0.328028, acc.: 85.94%] [G loss: 2.

305 [D loss: 0.760144, acc.: 45.31%] [G loss: 0.896174]
306 [D loss: 0.640382, acc.: 57.81%] [G loss: 1.040861]
307 [D loss: 0.783270, acc.: 43.75%] [G loss: 0.937871]
308 [D loss: 0.746750, acc.: 39.06%] [G loss: 0.999278]
309 [D loss: 0.704998, acc.: 51.56%] [G loss: 1.013111]
310 [D loss: 0.830623, acc.: 39.06%] [G loss: 0.802590]
311 [D loss: 0.753145, acc.: 45.31%] [G loss: 0.882331]
312 [D loss: 0.770288, acc.: 40.62%] [G loss: 0.833288]
313 [D loss: 0.729532, acc.: 53.12%] [G loss: 0.919056]
314 [D loss: 0.744655, acc.: 39.06%] [G loss: 0.844277]
315 [D loss: 0.803294, acc.: 37.50%] [G loss: 0.822090]
316 [D loss: 0.743526, acc.: 42.19%] [G loss: 0.833817]
317 [D loss: 0.856239, acc.: 31.25%] [G loss: 0.716514]
318 [D loss: 0.742582, acc.: 46.88%] [G loss: 0.756319]
319 [D loss: 0.777944, acc.: 39.06%] [G loss: 0.732335]
320 [D loss: 0.765123, acc.: 45.31%] [G loss: 0.800058]
321 [D loss: 0.754895, acc.: 40.62%] [G loss: 0.773487]
322 [D loss: 0.761823, acc.: 35.94%] [G loss: 0.

...

29156 [D loss: 0.647684, acc.: 60.94%] [G loss: 1.152811]
29157 [D loss: 0.609959, acc.: 65.62%] [G loss: 1.135486]
29158 [D loss: 0.601190, acc.: 65.62%] [G loss: 1.018935]
29159 [D loss: 0.675085, acc.: 57.81%] [G loss: 1.034133]
29160 [D loss: 0.641310, acc.: 60.94%] [G loss: 0.997222]
29161 [D loss: 0.666768, acc.: 62.50%] [G loss: 1.105294]
29162 [D loss: 0.594865, acc.: 70.31%] [G loss: 1.117565]
29163 [D loss: 0.712623, acc.: 64.06%] [G loss: 1.067194]
29164 [D loss: 0.619242, acc.: 67.19%] [G loss: 0.949945]
29165 [D loss: 0.582948, acc.: 65.62%] [G loss: 0.913906]
29166 [D loss: 0.670846, acc.: 51.56%] [G loss: 1.001519]
29167 [D loss: 0.582346, acc.: 67.19%] [G loss: 0.835217]
29168 [D loss: 0.572195, acc.: 75.00%] [G loss: 1.009966]
29169 [D loss: 0.657424, acc.: 64.06%] [G loss: 0.937406]
29170 [D loss: 0.584386, acc.: 67.19%] [G loss: 0.956203]
29171 [D loss: 0.639737, acc.: 62.50%] [G loss: 1.197576]
29172 [D loss: 0.536328, acc.: 73.44%] [G loss: 1.155457]
29173 [D loss:

29305 [D loss: 0.648721, acc.: 64.06%] [G loss: 0.993394]
29306 [D loss: 0.568589, acc.: 75.00%] [G loss: 0.925167]
29307 [D loss: 0.618040, acc.: 70.31%] [G loss: 1.020863]
29308 [D loss: 0.600952, acc.: 68.75%] [G loss: 1.137569]
29309 [D loss: 0.627186, acc.: 67.19%] [G loss: 1.020844]
29310 [D loss: 0.580950, acc.: 59.38%] [G loss: 1.042893]
29311 [D loss: 0.593830, acc.: 70.31%] [G loss: 1.016762]
29312 [D loss: 0.605059, acc.: 64.06%] [G loss: 0.929232]
29313 [D loss: 0.601145, acc.: 60.94%] [G loss: 1.038520]
29314 [D loss: 0.619674, acc.: 68.75%] [G loss: 1.146825]
29315 [D loss: 0.682744, acc.: 54.69%] [G loss: 1.139322]
29316 [D loss: 0.621459, acc.: 59.38%] [G loss: 0.952975]
29317 [D loss: 0.549647, acc.: 71.88%] [G loss: 0.914347]
29318 [D loss: 0.657888, acc.: 57.81%] [G loss: 1.002060]
29319 [D loss: 0.639130, acc.: 60.94%] [G loss: 1.071649]
29320 [D loss: 0.586666, acc.: 68.75%] [G loss: 1.112597]
29321 [D loss: 0.602490, acc.: 71.88%] [G loss: 1.037284]
29322 [D loss:

29448 [D loss: 0.549121, acc.: 67.19%] [G loss: 1.027228]
29449 [D loss: 0.694895, acc.: 54.69%] [G loss: 1.027190]
29450 [D loss: 0.590736, acc.: 67.19%] [G loss: 0.956407]
29451 [D loss: 0.651392, acc.: 64.06%] [G loss: 0.923890]
29452 [D loss: 0.549813, acc.: 76.56%] [G loss: 0.975594]
29453 [D loss: 0.685238, acc.: 57.81%] [G loss: 1.079248]
29454 [D loss: 0.650369, acc.: 62.50%] [G loss: 0.997268]
29455 [D loss: 0.594447, acc.: 70.31%] [G loss: 0.890233]
29456 [D loss: 0.667193, acc.: 56.25%] [G loss: 0.910950]
29457 [D loss: 0.546436, acc.: 78.12%] [G loss: 1.021963]
29458 [D loss: 0.705841, acc.: 56.25%] [G loss: 1.018484]
29459 [D loss: 0.600379, acc.: 68.75%] [G loss: 1.022060]
29460 [D loss: 0.701251, acc.: 50.00%] [G loss: 0.953661]
29461 [D loss: 0.599734, acc.: 71.88%] [G loss: 1.000811]
29462 [D loss: 0.571403, acc.: 75.00%] [G loss: 0.921690]
29463 [D loss: 0.652877, acc.: 65.62%] [G loss: 1.022985]
29464 [D loss: 0.692854, acc.: 50.00%] [G loss: 0.972646]
29465 [D loss:

29741 [D loss: 0.738100, acc.: 59.38%] [G loss: 0.981337]
29742 [D loss: 0.655809, acc.: 62.50%] [G loss: 1.012293]
29743 [D loss: 0.595337, acc.: 59.38%] [G loss: 1.011743]
29744 [D loss: 0.548009, acc.: 81.25%] [G loss: 1.070858]
29745 [D loss: 0.543648, acc.: 75.00%] [G loss: 1.065624]
29746 [D loss: 0.667969, acc.: 65.62%] [G loss: 1.034357]
29747 [D loss: 0.631024, acc.: 67.19%] [G loss: 1.101062]
29748 [D loss: 0.612713, acc.: 78.12%] [G loss: 1.009373]
29749 [D loss: 0.646872, acc.: 68.75%] [G loss: 1.132518]
29750 [D loss: 0.592161, acc.: 65.62%] [G loss: 0.946014]
29751 [D loss: 0.622343, acc.: 62.50%] [G loss: 1.114595]
29752 [D loss: 0.622060, acc.: 67.19%] [G loss: 1.047114]
29753 [D loss: 0.641986, acc.: 60.94%] [G loss: 1.055919]
29754 [D loss: 0.640247, acc.: 64.06%] [G loss: 1.064888]
29755 [D loss: 0.689115, acc.: 54.69%] [G loss: 1.151047]
29756 [D loss: 0.711708, acc.: 57.81%] [G loss: 1.105815]
29757 [D loss: 0.517268, acc.: 81.25%] [G loss: 1.055255]
29758 [D loss:

29891 [D loss: 0.682218, acc.: 51.56%] [G loss: 0.960156]
29892 [D loss: 0.620944, acc.: 64.06%] [G loss: 1.034436]
29893 [D loss: 0.606334, acc.: 70.31%] [G loss: 1.077382]
29894 [D loss: 0.575253, acc.: 73.44%] [G loss: 1.033326]
29895 [D loss: 0.606495, acc.: 68.75%] [G loss: 0.955881]
29896 [D loss: 0.606613, acc.: 68.75%] [G loss: 1.008964]
29897 [D loss: 0.581796, acc.: 75.00%] [G loss: 0.955177]
29898 [D loss: 0.643208, acc.: 67.19%] [G loss: 1.109297]
29899 [D loss: 0.723393, acc.: 51.56%] [G loss: 0.960304]
29900 [D loss: 0.593388, acc.: 67.19%] [G loss: 0.984987]
29901 [D loss: 0.614650, acc.: 60.94%] [G loss: 0.930924]
29902 [D loss: 0.641860, acc.: 54.69%] [G loss: 0.956902]
29903 [D loss: 0.631988, acc.: 67.19%] [G loss: 1.083968]
29904 [D loss: 0.663799, acc.: 65.62%] [G loss: 0.939571]
29905 [D loss: 0.616852, acc.: 65.62%] [G loss: 0.995395]
29906 [D loss: 0.615628, acc.: 65.62%] [G loss: 1.068037]
29907 [D loss: 0.607370, acc.: 64.06%] [G loss: 1.081003]
29908 [D loss:

In [2]:
#!mkdir images2

In [14]:
# yを入力として追加するConditionalGANへの拡張
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np
from keras import layers
from keras import backend as K

class ConditionalGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.y_size = 10

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        # labelを加える
        y = Input(shape=(self.y_size,))
        img = self.generator([z,y])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator([img, y])

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([z,y], validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        noise = Input(shape=(self.latent_dim,))
        # zは200yは1000
        z = layers.Dense(200, activation='relu')(noise)
        y_onehot = Input(shape=(self.y_size,))
        y = layers.Dense(1000, activation='relu')(y_onehot)
        gen_input = layers.concatenate([z, y],axis=1)
        dense_1 = Dense(256, activation=LeakyReLU(alpha=0.2))(gen_input)
        batch_1 = BatchNormalization(momentum=0.8)(dense_1)
        dense_2 = Dense(512, activation=LeakyReLU(alpha=0.2))(batch_1)
        batch_2 = BatchNormalization(momentum=0.8)(dense_2)
        dense_3 = Dense(1024, activation=LeakyReLU(alpha=0.2))(batch_2)
        batch_3 = BatchNormalization(momentum=0.8)(dense_3)
        dense_4 = Dense(np.prod(self.img_shape), activation='tanh')(batch_3)
        img = Reshape((self.img_shape))(dense_4)
        model = Model([noise, y_onehot], img)
        
        return model

    def build_discriminator(self):
        
        # xは240, yは50
        img = Input(shape=self.img_shape)
        flat = Flatten()(img)
        x = layers.Dense(240, activation='relu')(flat)
        y_onehot = Input(shape=(self.y_size,))
        y = layers.Dense(50, activation='relu')(y_onehot)
        dis_input = layers.concatenate([x,y],axis=1)
        dense_1 = Dense(512,activation=LeakyReLU(alpha=0.2))(dis_input)
        dense_2 = Dense(256,activation=LeakyReLU(alpha=0.2))(dense_1)
        validity = Dense(1,activation='sigmoid')(dense_2)
        
        #validity = (dis_input)
        model = Model([img,y_onehot], validity)
        return model

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        
        # One_Hot_Encoder_y_label
        y_onehot = K.one_hot(y_train, self.y_size)
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            y_batch = y_train[idx]
            labels = K.one_hot(y_batch, self.y_size)

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            

            # Generate a batch of new images
            # y_onehot？
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs,labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs,labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch([noise,y_onehot], valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        y =  K.one_hot(y_train[0], self.y_size)
        gen_imgs = self.generator.predict([noise, y])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images2/%d.png" % epoch)
        plt.close()



gan = ConditionalGAN()
gan.train(epochs=30000, batch_size=32,sample_interval=200)

C:\Users\umini\Anaconda3\envs\pytorch\lib\site-packages\keras\activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


ValueError: If your data is in the form of symbolic tensors, you should specify the `steps` argument (instead of the `batch_size` argument, because symbolic tensors are expected to produce batches of input data).